# Chapter 2. An Array of Sequences
---
## ToC

1. [Unpacking Sequences and Iterables](#unpacking-sequences-and-iterables)  
    1.1. [Using * to Grab Excess Items](#using-*-to-grab-excess-items)  
    1.2. [Unpacking with * in Function Calls and Sequence Literals](#unpacking-with-*-in-function-calls-and-sequence-literals)  
    1.3. [Nested Unpacking](#nested-unpacking)  
2. [Pattern Matching with Sequences](#pattern-matching-with-sequences)  
    2.1. [In an interpreter](#pattern-matching-sequences-in-an-interpreter)

In [1]:
from pathlib import Path
import sys

materials_path = Path().resolve() / "materials"
sys.path.insert(0, str(materials_path))

import lis # type: ignore

## Unpacking Sequences and Iterables

Unpacking is important because it avoids unnecessary and error-prone use of
indexes to extract elements from sequences. Also, unpacking works with any iterable
object as the data source—including iterators, which don’t support index notation
([]).

### Applications

**Parallel Assignment**

In [2]:
lax_coordinates = (33.9425, -118.408056)
latitude, longitude = lax_coordinates # unpacking
print(latitude)
print(longitude)

33.9425
-118.408056


**Swapping the Values of Variables**

In [3]:
a = 2
b = 3
b, a = a, b
print(a)
print(b)

3
2


**Refixing an Argument with * when Calling a Function**

In [4]:
divmod(20, 8)

(2, 4)

In [5]:
t = (20, 8)
divmod(*t)

(2, 4)

In [6]:
quotient, remainder = divmod(*t)
quotient, remainder

(2, 4)

**Return Multiple Values**

In [7]:
import os
_, filename = os.path.split('/home/Hamed/.ssh/id_rsa.pub')
filename

'id_rsa.pub'

### Using * to Grab Excess Items

Defining function parameters with `*args` to grab arbitrary excess arguments is a classic Python feature.

In [8]:
a, b, *rest = range(5)
a, b, rest

(0, 1, [2, 3, 4])

In [9]:
a, b, *rest = range(2)
a, b, rest

(0, 1, [])

In [10]:
*head, b, c, d = range(5)
head, b, c, d

([0, 1], 2, 3, 4)

In [11]:
a, *body, c, d = range(5)
a, body, c, d

(0, [1, 2], 3, 4)

### Unpacking with * in Function Calls and Sequence Literals

In [12]:
def fun(a, b, c, d, *rest):
    return a, b, c, d, rest

fun(*[1, 2], 3, *range(4, 7))

(1, 2, 3, 4, (5, 6))

**Defining `list`, `tuple`, or `set` Literals**

In [13]:
range(4)

range(0, 4)

In [14]:
*range(4), 4

(0, 1, 2, 3, 4)

In [15]:
[*range(4), 4]

[0, 1, 2, 3, 4]

In [16]:
{*range(4), 4, *(5, 6, 7)}

{0, 1, 2, 3, 4, 5, 6, 7}

### Nested Unpacking

In [17]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
    #,('Dummy') this entry causes error
]

def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for name, _, _, (lat, lon) in metro_areas:
        if lon <= 0:
            print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

if __name__ == '__main__':
    main()

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


The target of an unpacking assignment can also be a list, but good use cases are rare. If you have a database query that returns a single record (e.g., the SQL code has a LIMIT 1 clause), then you can unpack and at the same time make sure there’s only one result with this code

In [18]:
# [record] = query_returning_single_row()
# If the record has only one field, you can get it directly, like this:
# [[field]] = query_returning_single_row_with_single_field() 

## Pattern Matching with Sequences

In general, a sequence pattern matches the subject if:
1. The subject is a sequence and;
2. The subject and the pattern have the same number of items and;
3. Each corresponding item matches, including nested items.

Sequence patterns may be written as tuples or lists or any combination of nested
tuples and lists, but it makes no difference which syntax you use: in a sequence pattern,
square brackets and parentheses mean the same thing

![Figure 20](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/20.PNG)

[Structural Pattern Matching Documentation](https://fpy.li/2-6)

**match/case statement**

Imagine you are designing a robot that accepts commands sent as sequences of words and numbers, like `BEEPER
440 3`. After splitting into parts and parsing the numbers, you’d have a message like
`['BEEPER', 440, 3]`.

In [19]:
from typing import Union


class InvalidCommand(Exception):
    def __init__(self, message):
        self.message = message
        hint = (
            "Expected one of the following formats:\n"
            "  ['BEEPER', frequency:int, times:int]\n"
            "  ['NECK', angle:int]\n"
            "  ['LED', ident:str, intensity:int]\n"
            "  ['LED', ident:str, red:int, green:int, blue:int]"
        )
        super().__init__(f"Invalid command: {message}\n{hint}")


class LED:
    def __init__(self, ident: str):
        self.ident = ident
        self.color = (0, 0, 0)
        self.brightness = 0

    def set_brightness(self, intensity: int):
        self.brightness = intensity
        print(f"[LED {self.ident}] Brightness set to {intensity}")

    def set_color(self, red: int, green: int, blue: int):
        self.color = (red, green, blue)
        print(f"[LED {self.ident}] Color set to ({red}, {green}, {blue})")


class Neck:
    def __init__(self):
        self.angle = 0

    def rotate(self, angle: int):
        self.angle = angle
        print(f"[Neck] Rotated to {angle} degrees")


class BeepModule:
    def beep(self, times: int, frequency: int):
        print(f"[Beep] Beeping {times} times at {frequency} Hz")


class Robot:
    def __init__(self):
        self.leds = {
            "left": LED("left"),
            "right": LED("right"),
            "center": LED("center")
        }
        self.neck = Neck()
        self.beeper = BeepModule()

    def handle_command(self, message: list[str | int]):
        match message:
            case ['BEEPER', int(frequency), int(times)]:
                self.beeper.beep(times, frequency)
            case ['NECK', int(angle)]:
                self.neck.rotate(angle)
            case ['LED', str(ident), int(intensity)] if ident in self.leds:
                self.leds[ident].set_brightness(intensity)
            case ['LED', str(ident), int(r), int(g), int(b)] if ident in self.leds:
                self.leds[ident].set_color(r, g, b)
            case _:
                raise InvalidCommand(message)


In [20]:
robot = Robot()
robot.handle_command(['BEEPER', 440, 3])
robot.handle_command(['NECK', 90])
robot.handle_command(['LED', 'left', 128])
robot.handle_command(['LED', 'right', 255, 100, 50])

[Beep] Beeping 3 times at 440 Hz
[Neck] Rotated to 90 degrees
[LED left] Brightness set to 128
[LED right] Color set to (255, 100, 50)


In [21]:
robot.handle_command(['UNKNOWN'])  # raise InvalidCommand

InvalidCommand: Invalid command: ['UNKNOWN']
Expected one of the following formats:
  ['BEEPER', frequency:int, times:int]
  ['NECK', angle:int]
  ['LED', ident:str, intensity:int]
  ['LED', ident:str, red:int, green:int, blue:int]

**Destructuring**

One key improvement of `match` over `switch` is destructuring. Thanks to destructuring, pattern matching is a powerful tool to process records structured like nested mappings and sequences, which we often need to read from JSON APIs and databases with semi-structured schemas, like MongoDB, EdgeDB, or PostgreSQL.

In [22]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
    (2, 'BR', 19.649, (-23.547778, -46.635833)), # this entry is not correct but it's added
    ('Dummy') # this entry will be filtered out without causing error
]   
def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [name, _, _, (lat, lon)] if lon <= 0:
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

if __name__ == '__main__':
    main()                

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358
              2 |  -23.5478 |  -46.6358


![Figure 21](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/21.PNG)

```python
match tuple(phone):
    case ['1', *rest]: # North America and Caribbean
        ...
    case ['2', *rest]: # Africa and some territories
        ...
    case ['3' | '4', *rest]: # Europe
        ...
```                    

In the standard library, these types are compatible with sequence patterns:
```plaintext
list    memoryview  array.array
tuple   range       collections.deque
```

More pedantic patterns: adding types

```python 
case [str(name), _, _, (float(lat), float(lon)) as coord]:
```

if we want to match any subject sequence starting with a str, and ending with a nested sequence of two floats, we can write:

```python 
case [str(name), *_, (float(lat), float(lon))]:
```

In [23]:
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
    (2, 'BR', 19.649, (547778, -46.635833)), # type checking removes this entry
    ('Dummy') # we can add this thanks to sequence matching
]   
def main():
    print(f'{"":15} | {"latitude":>9} | {"longitude":>9}')
    for record in metro_areas:
        match record:
            case [str(name), _, _, (float(lat), float(lon)) as coord] if lon <= 0:
                print(f'{name:15} | {lat:9.4f} | {lon:9.4f}')

if __name__ == '__main__':
    main()                

                |  latitude | longitude
Mexico City     |   19.4333 |  -99.1333
New York-Newark |   40.8086 |  -74.0204
São Paulo       |  -23.5478 |  -46.6358


![Figure 22](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/22.PNG)

[Example: A very deep iterable and type match with extraction](https://fpy.li/2-10)

### Pattern Matching Sequences in an Interpreter

[lis.py](https://fpy.li/2-11): an interpreter for a subset of the Scheme dialect of the Lisp programming language in 132 lines of beautiful and readable Python code. 
The two main functions of lis.py are `parse` and `evaluate`.

In [24]:
lis.parse('(gcd 18 45)')

['gcd', 18, 45]

In [25]:
lis.parse('''
    (define double
    (lambda (n)
    (* n 2)))
    ''')

['define', 'double', ['lambda', ['n'], ['*', 'n', 2]]]

**Matching patterns without match/case**

```python
def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    
    if isinstance(exp, Symbol):  # variable reference
        return env[exp]
    
    # ... lines omitted

    elif exp[0] == 'quote':  # (quote exp)
        (_, x) = exp
        return x

    elif exp[0] == 'if':  # (if test conseq alt)
        (_, test, consequence, alternative) = exp
        if evaluate(test, env):
            return evaluate(consequence, env)
        else:
            return evaluate(alternative, env)

    elif exp[0] == 'lambda':  # (lambda (parm…) body…)
        (_, parms, *body) = exp
        return Procedure(parms, body, env)

    elif exp[0] == 'define':
        (_, name, value_exp) = exp
        env[name] = evaluate(value_exp, env)

    # ... more lines omitted
```

**Matching patterns with match/case**—requires Python ≥ 3.10

```python
def evaluate(exp: Expression, env: Environment) -> Any:
    "Evaluate an expression in an environment."
    
    match exp:
        # ... lines omitted

        case ['quote', x]:
            return x

        case ['if', test, consequence, alternative]:
            if evaluate(test, env):
                return evaluate(consequence, env)
            else:
                return evaluate(alternative, env)

        case ['lambda', [*parms], *body] if body:
            return Procedure(parms, body, env)

        case ['define', Symbol() as name, value_exp]:
            env[name] = evaluate(value_exp, env)

        # ... more lines omitted

        case _:
            raise SyntaxError(lispstr(exp))
```

For `lambda`:

```python
case ['lambda', [*parms], *body] if body:
    return Procedure(parms, body, env)
```    

**Note:** In a sequence pattern, `*` can appear only once per sequence. Here we have two
sequences: the outer and the inner.

![Figure 23](https://raw.githubusercontent.com/berserkhmdvhb/Training-Python/main/figures/Part_I/23.PNG)

**Further Reading**:

[(How to Write a (Lisp) Interpreter (in Python))](https://fpy.li/2-12)